In [1]:

import pandas as pd
import numpy as np
import main as fca
from sklearn.model_selection import train_test_split
from sklearn import preprocessing, metrics
from importlib import reload
import warnings
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
import lazy_pipeline as lpipe
import folium
from folium.plugins import HeatMap
import plotly.express as px
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,confusion_matrix,plot_roc_curve,classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')
%matplotlib inline
pd.set_option('display.max_columns', 32)

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
column_names = ['Sample code number','Clump Thickness','Uniformity of Cell Size',
        'Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size',
        'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class']
data = 'breast-cancer-wisconsin.data'
df = pd.read_csv(data, names=column_names)
df.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [3]:
(df['Bare Nuclei'] == '?').sum() #checking for rows with ? in it

16

In [4]:
df = df[df['Bare Nuclei'] != '?'] #dropping rows with ? present in it

In [5]:
df['Bare Nuclei'] = df['Bare Nuclei'].astype('int64') #we have all the features in numerical value.

In [6]:
df = df.drop(['Sample code number'], axis=1) #since we don't need code number so dropping it from the table
#df = df.select_dtypes(exclude=['object'])
df.head()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [7]:
df.loc[df['Class'] == 2, 'Class'] = 0 #updating target labels
df.loc[df['Class'] == 4, 'Class'] = 1

In [8]:
y_name='Class'
y = df[y_name]
X = lpipe.binarize_X(df.drop(y_name, axis=1))
print(X.shape)
X.head()

(683, 89)


,Clump Thickness: 1,Clump Thickness: 2,Clump Thickness: 3,Clump Thickness: 4,Clump Thickness: 5,Clump Thickness: 6,Clump Thickness: 7,Clump Thickness: 8,Clump Thickness: 9,Clump Thickness: 10,Uniformity of Cell Size: 1,Uniformity of Cell Size: 2,Uniformity of Cell Size: 3,Uniformity of Cell Size: 4,Uniformity of Cell Size: 5,Uniformity of Cell Size: 6,...,Normal Nucleoli: 4,Normal Nucleoli: 5,Normal Nucleoli: 6,Normal Nucleoli: 7,Normal Nucleoli: 8,Normal Nucleoli: 9,Normal Nucleoli: 10,Mitoses: 1,Mitoses: 2,Mitoses: 3,Mitoses: 4,Mitoses: 5,Mitoses: 6,Mitoses: 7,Mitoses: 8,Mitoses: 10
0,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False


In [9]:

X = X.sample(frac=1, random_state=42)  # Shuffle X
y = y[X.index]

In [24]:
import lazyfca as fca
reload(fca)

<module 'lazyfca' from 'd:\\HSE\\OSDA\\LazyFCA\\lazyfca.py'>

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

In [14]:
results = fca.predict(X_train, X_test, y_train, y_test)
#results

In [23]:
fca_results = fca.classificationReport(results)
fca_results

,Accuracy,Precision,Recall
0,92.195122,100.0,78.947368


In [38]:
#Accuracy
fca_acc = fca_results['Accuracy'][0]
fca_acc

92.19512195121952

In [35]:
#Let's dive in doing the ML model


92.19512195121952

Decision Tree Classifier

In [25]:
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)

y_pred_dtc = dtc.predict(X_test)

acc_dtc = accuracy_score(y_test, y_pred_dtc)
conf = confusion_matrix(y_test, y_pred_dtc)
clf_report = classification_report(y_test, y_pred_dtc)

print(f"Accuracy Score of Decision Tree is : {acc_dtc}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

Accuracy Score of Decision Tree is : 0.9219512195121952
Confusion Matrix : 
[[125   4]
 [ 12  64]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.91      0.97      0.94       129
           1       0.94      0.84      0.89        76

    accuracy                           0.92       205
   macro avg       0.93      0.91      0.91       205
weighted avg       0.92      0.92      0.92       205



Random Forest Classifier

In [26]:
rd_clf = RandomForestClassifier()
rd_clf.fit(X_train, y_train)

y_pred_rd_clf = rd_clf.predict(X_test)

acc_rd_clf = accuracy_score(y_test, y_pred_rd_clf)
conf = confusion_matrix(y_test, y_pred_rd_clf)
clf_report = classification_report(y_test, y_pred_rd_clf)

print(f"Accuracy Score of Random Forest is : {acc_rd_clf}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

Accuracy Score of Random Forest is : 0.9707317073170731
Confusion Matrix : 
[[128   1]
 [  5  71]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.96      0.99      0.98       129
           1       0.99      0.93      0.96        76

    accuracy                           0.97       205
   macro avg       0.97      0.96      0.97       205
weighted avg       0.97      0.97      0.97       205



Ada Boost Classifier

In [27]:
ada = AdaBoostClassifier(base_estimator = dtc)
ada.fit(X_train, y_train)

y_pred_ada = ada.predict(X_test)

acc_ada = accuracy_score(y_test, y_pred_ada)
conf = confusion_matrix(y_test, y_pred_ada)
clf_report = classification_report(y_test, y_pred_ada)

print(f"Accuracy Score of Ada Boost Classifier is : {acc_ada}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

Accuracy Score of Ada Boost Classifier is : 0.9365853658536586
Confusion Matrix : 
[[125   4]
 [  9  67]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.93      0.97      0.95       129
           1       0.94      0.88      0.91        76

    accuracy                           0.94       205
   macro avg       0.94      0.93      0.93       205
weighted avg       0.94      0.94      0.94       205



Gradient Boosting Classifier

In [28]:
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

y_pred_gb = gb.predict(X_test)

acc_gb = accuracy_score(y_test, y_pred_gb)
conf = confusion_matrix(y_test, y_pred_gb)
clf_report = classification_report(y_test, y_pred_gb)

print(f"Accuracy Score of Ada Boost Classifier is : {acc_gb}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

Accuracy Score of Ada Boost Classifier is : 0.9658536585365853
Confusion Matrix : 
[[128   1]
 [  6  70]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.96      0.99      0.97       129
           1       0.99      0.92      0.95        76

    accuracy                           0.97       205
   macro avg       0.97      0.96      0.96       205
weighted avg       0.97      0.97      0.97       205



XgBoost Classifier

In [29]:
xgb = XGBClassifier(booster = 'gbtree', learning_rate = 0.1, max_depth = 5, n_estimators = 180)
xgb.fit(X_train, y_train)

y_pred_xgb = xgb.predict(X_test)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
conf = confusion_matrix(y_test, y_pred_xgb)
clf_report = classification_report(y_test, y_pred_xgb)

print(f"Accuracy Score of Ada Boost Classifier is : {acc_xgb}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

Accuracy Score of Ada Boost Classifier is : 0.9609756097560975
Confusion Matrix : 
[[126   3]
 [  5  71]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       129
           1       0.96      0.93      0.95        76

    accuracy                           0.96       205
   macro avg       0.96      0.96      0.96       205
weighted avg       0.96      0.96      0.96       205



Cat Boost Classifier

In [30]:
cat = CatBoostClassifier(iterations=10)
cat.fit(X_train, y_train)

y_pred_cat = cat.predict(X_test)

acc_cat = accuracy_score(y_test, y_pred_cat)
conf = confusion_matrix(y_test, y_pred_cat)
clf_report = classification_report(y_test, y_pred_cat)
print(f"Accuracy Score of Ada Boost Classifier is : {acc_cat}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

Learning rate set to 0.5
0:	learn: 0.2358679	total: 234ms	remaining: 2.11s
1:	learn: 0.1559588	total: 256ms	remaining: 1.02s
2:	learn: 0.1022156	total: 265ms	remaining: 617ms
3:	learn: 0.0843031	total: 272ms	remaining: 408ms
4:	learn: 0.0723687	total: 279ms	remaining: 279ms
5:	learn: 0.0601326	total: 283ms	remaining: 188ms
6:	learn: 0.0555521	total: 292ms	remaining: 125ms
7:	learn: 0.0490796	total: 295ms	remaining: 73.7ms
8:	learn: 0.0441883	total: 306ms	remaining: 34ms
9:	learn: 0.0374694	total: 310ms	remaining: 0us
Accuracy Score of Ada Boost Classifier is : 0.975609756097561
Confusion Matrix : 
[[126   3]
 [  2  74]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       129
           1       0.96      0.97      0.97        76

    accuracy                           0.98       205
   macro avg       0.97      0.98      0.97       205
weighted avg       0.98      0.98      0.98       205



Extra Trees Classifier

In [31]:
etc = ExtraTreesClassifier()
etc.fit(X_train, y_train)

y_pred_etc = etc.predict(X_test)

acc_etc = accuracy_score(y_test, y_pred_etc)
conf = confusion_matrix(y_test, y_pred_etc)
clf_report = classification_report(y_test, y_pred_etc)

print(f"Accuracy Score of Ada Boost Classifier is : {acc_etc}")
print(f"Confusion Matrix : \n{conf}")
print(f"Classification Report : \n{clf_report}")

Accuracy Score of Ada Boost Classifier is : 0.975609756097561
Confusion Matrix : 
[[127   2]
 [  3  73]]
Classification Report : 
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       129
           1       0.97      0.96      0.97        76

    accuracy                           0.98       205
   macro avg       0.98      0.97      0.97       205
weighted avg       0.98      0.98      0.98       205



Models Comparison

In [41]:
models = pd.DataFrame({
    'Model' : ['Decision Tree Classifier', 'Random Forest Classifier','Ada Boost Classifier',
             'Gradient Boosting Classifier', 'XgBoost', 'Cat Boost', 'Extra Trees Classifier', 'Fca Classifier'],
    'Score' : [acc_dtc, acc_rd_clf, acc_ada, acc_gb, acc_xgb, acc_cat, acc_etc, fca_acc/100]
})


models.sort_values(by = 'Score', ascending = False)

,Model,Score
5,Cat Boost,0.975610
6,Extra Trees Classifier,0.975610
1,Random Forest Classifier,0.970732
3,Gradient Boosting Classifier,0.965854
4,XgBoost,0.960976
2,Ada Boost Classifier,0.936585
0,Decision Tree Classifier,0.921951
7,Fca Classifier,0.921951


In [42]:
px.bar(data_frame = models, x = 'Score', y = 'Model', color = 'Score', template = 'plotly_dark', title = 'Models Comparison')